Copyright (C) Microsoft Corporation.

# Load Model and Detect Out of Stock

In this notebook, we adapt the inferancing steps provided [here](https://github.com/tensorflow/models/blob/master/research/object_detection/object_detection_tutorial.ipynb) to detect out-of-stock in the validation set and save the results.

In [1]:
import numpy as np
import os
import sys
import tensorflow as tf
import glob

from distutils.version import StrictVersion
from collections import defaultdict
from PIL import Image
from utilities import create_dir, load_image_into_numpy_array
from IPython.display import display

sys.path.append("../models/research/") # Modify to point to your tensorflow models directory
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

if StrictVersion(tf.__version__) < StrictVersion('1.12.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.12.*.')
print(tf.__version__)

1.12.0


# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_FROZEN_GRAPH` to point to a new .pb file. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [2]:
OUT_IMAGES = './val_maskrcnn'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = './export_maskrcnn/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = './stockout_label_map.pbtxt'

data_dir = './annotated/'

In [3]:
create_dir(OUT_IMAGES)

## Load a (frozen) Tensorflow model into memory.

In [4]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [5]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

# Detection

In [6]:
import random
import glob
random.seed(4)

In [7]:
in_images = glob.glob(data_dir + '/**/'+ '*.jpg')
print("There are {} images.".format(len(in_images)))

There are 538 images.


In [8]:
in_images_sub = glob.glob(data_dir + '/**/**/'+ '*.jpg')
print("There are {} images.".format(len(in_images_sub)))

There are 239 images.


In [9]:
in_images = in_images + in_images_sub
print("There are {} images.".format(len(in_images)))

There are 777 images.


In [10]:
# in_images = glob.glob(data_dir + '*.jpg')
# print("There are {} images.".format(len(in_images)))

In [11]:
num_val = int(len(in_images) * 0.1)
val_images = random.sample(in_images, num_val)

In [12]:
len(val_images)

77

In [13]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

In [14]:
# Size, in inches, of the output images.
IMAGE_SIZE = (24, 18)

In [15]:
%%time

for image_path in val_images:
    image = Image.open(image_path)
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = load_image_into_numpy_array(image)
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    # Actual detection.
    output_dict = run_inference_for_single_image(image_np, detection_graph)
    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        instance_masks=output_dict.get('detection_masks'),
        use_normalized_coordinates=True,
        line_thickness=4)
    image_np = Image.fromarray(image_np)
    save_path = os.path.split(image_path)[-1]
    image_np.save(os.path.join(OUT_IMAGES, save_path))
    print(save_path)

1555542176.5557559.jpg
1555961293.7713525.jpg
1555539787.0466478.jpg
1555801921.4971292.jpg
1555976568.4544413.jpg
1555456504.342944.jpg
1555539975.0110133.jpg
1555445519.086689.jpg
1555445561.6098683.jpg
1555540839.013019.jpg
1555976356.0379443.jpg
1555801863.5133677.jpg
1555394416.5971968.jpg
1555445585.7518868.jpg
1555542066.6238005.jpg
1555456068.1823683.jpg
1555802057.5336428.jpg
1555976748.0456614.jpg
1555394720.9561694.jpg
1555539165.7225947.jpg
1555538997.3746176.jpg
1555394700.4720445.jpg
1555541663.2072036.jpg
1555540729.746675.jpg
1555802093.2590091.jpg
1555394495.9759912.jpg
1555394577.7752154.jpg
1555541744.524736.jpg
1555539690.686179.jpg
1555961300.4531832.jpg
1555801871.3186634.jpg
1555802215.9086375.jpg
1555976541.6671298.jpg
1555445539.6911302.jpg
1555802028.5732923.jpg
1555960898.444229.jpg
1555802189.1586087.jpg
1555976994.7344933.jpg
1555456016.267854.jpg
1555394543.6131394.jpg
1555538985.927765.jpg
1555394982.938294.jpg
1555456139.866951.jpg
1555394374.7426455.jpg

Let's check some of these images.

In [16]:
val_images_detect = glob.glob(OUT_IMAGES + '/*.jpg')

In [1]:
# for img_path in val_images_detect:
#     img = Image.open(img_path)
#     display(img)
#     print(img.size)